In [1]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
data_url = 'df_'
train_data = TabularDataset(f'{data_url}train.csv')
train_data.head()

,Dst Port,Protocol,Fwd Pkt Len Std,Bwd Pkt Len Min,Flow Byts/s,Fwd IAT Mean,Bwd IAT Tot,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,...,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Std,Active Max,Active Min,Idle Max,Idle Min,Label
0,0.506211,0.352941,0.026531,0.0,3.702185e-07,0.898484,0.000149,0.000113,0.000131,0.000018,...,0.003632,0.003235,0.000004,0.666667,0.0,0.0,0.0,0.0,0.0,Benign
1,0.006760,0.352941,0.000000,0.0,0.000000e+00,0.897540,0.000000,0.000000,0.000000,0.000000,...,0.979767,0.000000,0.000000,0.416667,0.0,0.0,0.0,0.0,0.0,Benign
2,0.123295,0.352941,0.000000,0.0,0.000000e+00,0.897541,0.000000,0.000000,0.000000,0.000000,...,0.003922,0.000000,0.000000,0.416667,0.0,0.0,0.0,0.0,0.0,Botnet
3,0.000320,0.352941,0.000000,0.0,0.000000e+00,0.897540,0.000000,0.000000,0.000000,0.000000,...,0.410217,0.000015,0.000000,0.833333,0.0,0.0,0.0,0.0,0.0,Brute-force
4,0.000320,0.352941,0.000000,0.0,0.000000e+00,0.897540,0.000000,0.000000,0.000000,0.000000,...,0.410217,0.000015,0.000000,0.833333,0.0,0.0,0.0,0.0,0.0,Brute-force


In [4]:
label = 'Label'
train_data[label].describe()

count     3623992
unique          7
top        Benign
freq      1811996
Name: Label, dtype: object

In [5]:
save_path = 'model_save_path/'  # specifies folder to store trained models
predictor = TabularPredictor(label=label).fit(train_data, num_gpus=1, hyperparameters={
    'CAT':{}, 'XGB':{},
    'RF':{},
    'KNN':{}
})

No path specified. Models will be saved in: "AutogluonModels\ag-20240330_131859"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240330_131859"
=================== System

In [6]:
test_data = TabularDataset(f'{data_url}test.csv')

y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

Loaded data from: df_test.csv | Columns = 35 / 35 | Rows = 905998 -> 905998


0         Benign
1         Benign
2    DDoS attack
3         Botnet
4         Benign
Name: Label, dtype: object

In [7]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9549402978814523,
 'balanced_accuracy': 0.8458093867492769,
 'mcc': 0.933941927521142}

In [8]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.954940,0.955436,accuracy,2494.668355,97.279417,105.380653,0.149019,0.002999,2.220671,2,True,4
1,XGBoost,0.954752,0.955381,accuracy,15.720550,0.081143,71.117085,15.720550,0.081143,71.117085,1,True,3
2,CatBoost,0.953785,0.954498,accuracy,0.627692,0.025026,27.375011,0.627692,0.025026,27.375011,1,True,2
3,KNeighbors,0.951580,0.951959,accuracy,2478.171094,97.170248,4.667887,2478.171094,97.170248,4.667887,1,True,1
